In [14]:
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.ddpg import DDPGTrainer
import ray
import shared_constants
import tensorflow as tf
from ray.rllib.contrib.maddpg import MADDPGTrainer
from ray.tune import register_env
from gym_pybullet_drones.envs.multi_agent_rl.PayloadCoop import PayloadCoop
from ray.rllib.agents.callbacks import DefaultCallbacks
import shared_constants
from gym_pybullet_drones.envs.single_agent_rl.BaseSingleAgentAviary import ActionType, ObservationType

In [15]:
import os
import time
import argparse
from datetime import datetime
import subprocess
import pdb
import math
import numpy as np
import pybullet as p
import pickle
import matplotlib.pyplot as plt
import gym
from gym import error, spaces, utils
from gym.utils import seeding
from gym.spaces import Box, Dict
import torch
import torch.nn as nn
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork
import ray
from ray import tune
from ray.tune.logger import DEFAULT_LOGGERS
from ray.tune import register_env
from ray.rllib.agents import ppo, ddpg
from ray.rllib.agents.ppo import PPOTrainer, PPOTFPolicy
from ray.rllib.examples.policy.random_policy import RandomPolicy
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.callbacks import DefaultCallbacks
from ray.rllib.models.modelv2 import ModelV2
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.sample_batch import SampleBatch
from ray.rllib.env.multi_agent_env import ENV_STATE
from ray.rllib.utils.annotations import override
from gym_pybullet_drones.envs.BaseAviary import DroneModel, Physics
from gym_pybullet_drones.envs.multi_agent_rl.FlockAviary import FlockAviary
from gym_pybullet_drones.envs.multi_agent_rl.LeaderFollowerAviary import LeaderFollowerAviary
from gym_pybullet_drones.envs.multi_agent_rl.MeetupAviary import MeetupAviary
from gym_pybullet_drones.envs.multi_agent_rl.PayloadCoop import PayloadCoop
from gym_pybullet_drones.envs.single_agent_rl.BaseSingleAgentAviary import ActionType, ObservationType
from gym_pybullet_drones.utils.Logger import Logger

from ray.rllib.models.torch.misc import SlimFC, AppendBiasLayer, normc_initializer

import shared_constants
import tensorflow as tf
import tensorflow.keras.layers as layers
from gym_pybullet_drones.envs.single_agent_rl.BaseSingleAgentAviary import ActionType, ObservationType
from pprint import PrettyPrinter



#### Useful links ##########################################
# Workflow: github.com/ray-project/ray/blob/master/doc/source/rllib-training.rst
# ENV_STATE example: github.com/ray-project/ray/blob/master/rllib/examples/env/two_step_game.py
# Competing policies example: github.com/ray-project/ray/blob/master/rllib/examples/rock_paper_scissors_multiagent.py
############################################################
class CustomTorchCentralizedCriticModel(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        # self.action_model = nn.Sequential(
        #     # nn.BatchNorm1d(OWN_OBS_VEC_SIZE),
        #     nn.Linear(OWN_OBS_VEC_SIZE, 64),
        #     nn.Tanh(),
        #     nn.Linear(64, 64),
        #     nn.Tanh(),
        #     nn.Linear(64, num_outputs),
        #     nn.Tanh()
        # )
        # self.value_model = nn.Sequential(
        #     # nn.BatchNorm1d(obs_space.shape[0]),
        #     nn.Linear(obs_space.shape[0], 64),
        #     nn.ReLU(),
        #     nn.Linear(64, 64),
        #     nn.ReLU(),
        #     nn.Linear(64, 64),
        #     nn.ReLU(),
        #     nn.Linear(64, 1),
        # )
        self.policy_model = action_model
        self.q_model = value_model
        self._model_in = None

    def forward(self, input_dict, state, seq_lens):
        self._model_in = [input_dict["obs_flat"], state, seq_lens]
        # return self.action_model({"obs": input_dict["obs"]["own_obs"]}, state, seq_lens)
        obs = input_dict["obs"]["own_obs"].float()
        print(input_dict)
#         opp_obs = input_dict["obs"]["opponent_obs"].float()
#         print("obs: {}".format(obs))
#         print("opp_obs: {}".format(opp_obs))
        self._last_flat_in = obs.reshape(obs.shape[0], -1)
        self._features = self.policy_model(self._last_flat_in)
        return self._features, state

    def value_function(self):
        value_out, _ = self.q_model({"obs": self._model_in[0]}, self._model_in[1], self._model_in[2])
        return torch.reshape(value_out, [-1])


############################################################
current_phase=0
class FillInActions(DefaultCallbacks):
    def on_postprocess_trajectory(self, worker, episode, agent_id, policy_id, policies, postprocessed_batch, original_batches, **kwargs):
        to_update = postprocessed_batch[SampleBatch.CUR_OBS]
        other_id = 1 if agent_id == 0 else 0
        action_encoder = ModelCatalog.get_preprocessor_for_space( 
                                                                 Box(-np.inf, np.inf, (ACTION_VEC_SIZE,), np.float32) # Unbounded
                                                                 )
        # action_encoder = ModelCatalog.get_preprocessor_for_space( 
        #                                                          Discrete(ACTION_VEC_SIZE) # Unbounded
        #                                                          )
        _, opponent_batch = original_batches[other_id]
        opponent_actions = np.array([action_encoder.transform(a) for a in opponent_batch[SampleBatch.ACTIONS]])
        to_update[:, -ACTION_VEC_SIZE:] = opponent_actions

    def on_train_result(self, trainer, result):
        global current_phase
        print("Manage Curriculum callback called on phase {}".format(current_phase))
        if result["episode_reward_mean"] > 0:
            current_phase+=1
            trainer.workers.foreach_worker(
            lambda ev: ev.foreach_env(
                lambda env: env.set_phase(current_phase)))


############################################################
def central_critic_observer(agent_obs, **kw):
    new_obs = {
        0: {
            "own_obs": agent_obs[0],
            "opponent_obs": agent_obs[1],
            "opponent_action": np.zeros(ACTION_VEC_SIZE), # Filled in by FillInActions
        },
        1: {
            "own_obs": agent_obs[1],
            "opponent_obs": agent_obs[0],
            "opponent_action": np.zeros(ACTION_VEC_SIZE), # Filled in by FillInActions
        },
    }
    return new_obs



#### Save directory ########################################
# filename = os.path.dirname(os.path.abspath(__file__))+'/results/save-'+ARGS.env+'-'+str(ARGS.num_drones)+'-'+ARGS.algo+'-'+ARGS.obs.value+'-'+ARGS.act.value+'-'+datetime.now().strftime("%m.%d.%Y_%H.%M.%S")
# if not os.path.exists(filename):
#     os.makedirs(filename+'/')

class ARGS:
    num_drones = shared_constants.NUM_DRONES
    env = shared_constants.ENV
    obs = ObservationType(shared_constants.OBS)
    act = ActionType(shared_constants.ACT)
    algo = 'cc'
    workers = 7
    
#### Initialize Ray Tune ###################################
ray.shutdown()
ray.init(ignore_reinit_error=True)



#### Register the environment ##############################
temp_env_name = "this-aviary-v0" 
register_env(temp_env_name, lambda _: PayloadCoop(num_drones=ARGS.num_drones,
                                                    aggregate_phy_steps=shared_constants.AGGR_PHY_STEPS,
                                                    obs=ARGS.obs,
                                                    act=ARGS.act
                                                    ))
temp_env = PayloadCoop(num_drones=ARGS.num_drones,
                            aggregate_phy_steps=shared_constants.AGGR_PHY_STEPS,
                            obs=ARGS.obs,
                            act=ARGS.act,
                            )
observer_space = Dict({
    "own_obs": temp_env.observation_space[0],
    "opponent_obs": temp_env.observation_space[0],
    "opponent_action": temp_env.action_space[0],
})
action_space = temp_env.action_space[0]
OWN_OBS_VEC_SIZE = observer_space["own_obs"].shape[0]
ACTION_VEC_SIZE = action_space.shape[0]
action_model = nn.Sequential(
                # nn.BatchNorm1d(OWN_OBS_VEC_SIZE),
                nn.Linear(OWN_OBS_VEC_SIZE + 2, 64),
                nn.Tanh(),
                nn.Linear(64, 64),
                nn.Tanh(),
                nn.Linear(64, 5),
                nn.Tanh()
            )
value_model =  nn.Sequential(
                # nn.BatchNorm1d(obs_space.shape[0]),
                nn.Linear(2*OWN_OBS_VEC_SIZE + 1, 64),
                nn.ReLU(),
                nn.Linear(64, 64),
                nn.ReLU(),
                nn.Linear(64, 64),
                nn.ReLU())
ModelCatalog.register_custom_model("cc_model", CustomTorchCentralizedCriticModel)
# ModelCatalog.register_custom_model("cc_model", TFModelCC)
#### Note ##################################################
# RLlib will create ``num_workers + 1`` copies of the
# environment since one copy is needed for the driver process.
# To avoid paying the extra overhead of the driver copy,
# which is needed to access the env's action and observation spaces,
# you can defer environment initialization until ``reset()`` is called

#### Set up the trainer's config ###########################
# config = ddpg.DEFAULT_CONFIG.copy() # For the default config, see github.com/ray-project/ray/blob/master/rllib/agents/trainer.py
config = {
    "env": temp_env_name,
    "num_workers": 0 + ARGS.workers,
    "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")), # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0
    "batch_mode": "complete_episodes",
    "callbacks":FillInActions,
    "framework": "torch",
    "lr" : shared_constants.LR,
    "gamma": shared_constants.GAMMA,
    "n_step": shared_constants.N_STEP,
    "num_envs_per_worker": 2,
    "in_evaluation": True,
    "explore": False,
    # "batch_mode": "complete_episodes",
    # "train_batch_size":200,
    # "rollout_fragment_length":200,
}

#### Set up the model parameters of the trainer's config ###
config["model"] = { 
    "custom_model": "cc_model",
}

#### Set up the multiagent params of the trainer's config ##
config["multiagent"] = { 
    "policies": {
        "pol0": (None, observer_space, action_space, {"agent_id": 0,}),
        "pol1": (None, observer_space, action_space, {"agent_id": 1,}),
    },
    "policy_mapping_fn": lambda x: "pol0" if x == 0 else "pol1", # # Function mapping agent ids to policy ids
    "observation_fn": central_critic_observer, # See rllib/evaluation/observation_function.py for more info
}

#### Ray Tune stopping conditions ##########################
stop = {
    "timesteps_total": 3, # 8000,
    # "episode_reward_mean": 0,
    # "training_iteration": 0,
}


2021-03-25 13:25:30,596	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8265


[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 0.000000, km 0.000000,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000


In [16]:
tr = DDPGTrainer(config = config)
pl0 = tr.get_policy("pol0")
pl1 = tr.get_policy("pol1")

2021-03-25 13:25:37,861	WARNING ddpg.py:156 -- Setting use_state_preprocessor=True since a custom model was specified.
(pid=28957) WARNING:tensorflow:From /home/mahendra/anaconda3/envs/TA/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=28957) Instructions for updating:
(pid=28957) non-resource variables are not supported in the long term
(pid=28959) WARNING:tensorflow:From /home/mahendra/anaconda3/envs/TA/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=28959) Instructions for updating:
(pid=28959) non-resource variables are not supported in the long term
(pid=28955) WARNING:tensorflow:From /home/mahendra/anaconda3/envs/TA/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.p

(pid=28955) [INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
(pid=28955) [INFO] m 0.027000, L 0.039700,
(pid=28955) [INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
(pid=28955) [INFO] kf 0.000000, km 0.000000,
(pid=28955) [INFO] t2w 2.250000, max_speed_kmh 30.000000,
(pid=28955) [INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
(pid=28955) [INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
(pid=28955) [INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000
(pid=28961) [INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
(pid=28961) [INFO] m 0.027000, L 0.039700,
(pid=28961) [INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
(pid=28961) [INFO] kf 0.000000, km 0.000000,
(pid=28961) [INFO] t2w 2.250000, max_speed_kmh 30.000000,
(pid=28961) [INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
(pid=28961) [INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
(pid=28961) [INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coef

(pid=28959) /home/mahendra/anaconda3/envs/TA/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=28959)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=28955) /home/mahendra/anaconda3/envs/TA/lib/python3.6/site-packages/ray/rllib/agents/ddpg/ddpg_torch_model.py:55: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
(pid=28955)   torch.from_numpy(self.action_space.low).float())
(pid=28961) /home/mahendra/anaconda3/envs/TA/lib/python3.6/site-packages/ray/rllib/agents/ddpg/ddpg_torch_model.py:55: User

(pid=28955) {'obs': OrderedDict([('opponent_action', tensor([[0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0., 0.],
(pid=28955)         [0., 0., 0., 0.

(pid=28960)         [0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28960)         [0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28960)         [0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28960)         [0., 0., 0., 0., 0., 0., 0., 0.]]))]), 'is_training': False, 'obs_flat': tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28960)         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28960)         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28960)         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28960)         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28960)         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28960)         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28962) {'o

RayTaskError(RuntimeError): [36mray::RolloutWorker.foreach_policy()[39m (pid=28962, ip=192.168.100.43)
  File "python/ray/_raylet.pyx", line 422, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 456, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 459, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 463, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 415, in ray._raylet.execute_task.function_executor
  File "/home/mahendra/anaconda3/envs/TA/lib/python3.6/site-packages/ray/rllib/evaluation/rollout_worker.py", line 463, in __init__
    policy_dict, policy_config)
  File "/home/mahendra/anaconda3/envs/TA/lib/python3.6/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1076, in _build_policy_map
    policy_map[name] = cls(obs_space, act_space, merged_conf)
  File "/home/mahendra/anaconda3/envs/TA/lib/python3.6/site-packages/ray/rllib/policy/torch_policy_template.py", line 251, in __init__
    stats_fn=stats_


(pid=28959)         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28959)         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28959)         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28959)         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28959)         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28959)         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28959)         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28959)         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28959)         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
(pid=28959)         [0., 0., 0., 0., 0., 0., 

In [ ]:
a = np.hstack([temp_env.observation_space[0].sample(), temp_env.observation_space[0].sample(), temp_env.action_space[0].sample()])

In [ ]:
a_t = torch.FloatTensor(temp_env.observation_space[0].sample())
b_t = torch.FloatTensor(action_space.sample())

(pid=12559) /home/mahendra/anaconda3/envs/TA/lib/python3.6/site-packages/ray/rllib/agents/ddpg/ddpg_torch_model.py:55: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
(pid=12559)   torch.from_numpy(self.action_space.low).float())


(pid=12563) {'obs': OrderedDict([('opponent_action', tensor([[0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0., 0.],
(pid=12563)         [0., 0., 0., 0.

In [92]:
a_t.shape

torch.Size([8])

In [106]:
pl1.model

CustomTorchCentralizedCriticModel_as_DDPGTorchModel(
  (action_model): Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=256, bias=True)
    (5): Tanh()
  )
  (value_model): Sequential(
    (0): Linear(in_features=17, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
  )
  (policy_model): Sequential(
    (action_0): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=400, bias=True)
        (1): ReLU()
      )
    )
    (action_1): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=400, out_features=300, bias=True)
        (1): ReLU()
      )
    )
    (action_out): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=300,

In [ ]:
import numpy as np
max_ = np.zeros(5)
min_ = np.zeros(5)
val_max = 0
val_min = 0
for i in range(5000):
    act = pl.compute_single_action(np.hstack([temp_env.observation_space[0].sample(), temp_env.observation_space[0].sample(), temp_env.action_space[0].sample()]), policy_id = 'pol1')[0]
    value = pl.model.value_function()
    value = value.detach().numpy()
    for i in range(5):
        if(act[i] > max_[i]):
            max_[i] = act[i]
        if(act[i] < min_[i]):
            min_[i] = act[i]
        if(value > val_max):
            val_max = value
        if (value < val_min):
            val_min = value

In [111]:
ray.shutdown()

In [11]:
tf.__version__

'2.3.0'

In [13]:
ray.shutdown()

In [29]:
class parent:
    def __init__(self):
        self.x = 5
    def a(self):
        print(self.x)
        
    def b(self):
        print(self.x)

In [33]:
class child(parent):
    def __init__(self):
        self.x = 12
    @override(parent)
    def a(self):
        print(1/0)
    